

# Este modelo de red entrena un BiLSTM + CRF para la clasificación de NER sobre el corpus Conll2002. Este modelo tiene como entrada a la red la enterización del conjunto X de entrenamiento y la enterización y categorización  de los vectores de etiquetas. 

In [36]:
from multiprocessing import cpu_count

print(cpu_count())

24


In [1]:
try:

    import seqeval

except ModuleNotFoundError as err:

    !pip install seqeval

In [38]:
import tensorflow as tf

#matplotlib inline

import matplotlib.pyplot as plt

plt.style.use('ggplot')

import os

for dirname, _, filenames in os.walk('inputs'):

    for filename in filenames:

        print(os.path.join(dirname, filename))

inputs\ccfasttext\cc.es.300.vec
inputs\ccfasttext\word2vec_skip-gram_model_300.txt
inputs\libs2021\mwrapper.py
inputs\libs2021\tf2crf.py
inputs\libs2021\utils.py
inputs\libs2021\__pycache__\mwrapper.cpython-310.pyc
inputs\libs2021\__pycache__\tf2crf.cpython-310.pyc
inputs\libs2021\__pycache__\utils.cpython-310.pyc
inputs\libscrf4\crfta.py
inputs\libscrf4\utils.py


In [39]:
import sys

sys.path.append('inputs/libs2021')

#sys.path.append('/inputs/embedding')

sys.path.append('inputs/libscrf4')



import tensorflow as tf

import matplotlib.pyplot as plt

import numpy as np

import pickle

import pandas as pd

from itertools import islice



from tabulate import tabulate

from sklearn.metrics import confusion_matrix

from sklearn.metrics import classification_report as eskclarep

#from seqeval.metrics import precision_score, recall_score, f1_score, accuracy_score

#from seqeval.metrics import classification_report as seqclarep

from sklearn.preprocessing import LabelBinarizer

from itertools import chain



from tensorflow.keras.preprocessing.sequence import pad_sequences

from tensorflow.keras.models import Model, Sequential

from tensorflow.keras.layers import Concatenate, Lambda, Input, LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional, InputLayer, Activation, Flatten

from tensorflow.keras.optimizers import Adam, schedules

#from crfta import CRF as crf4

from utils import build_matrix_embeddings as bme, plot_model_performance, logits_to_tokens, report_to_df



from tensorflow.keras.callbacks import ModelCheckpoint

from tensorflow.keras.callbacks import TensorBoard



from IPython.core.display import display, HTML



import datetime, os

import random




C:\Users\Univalle\AppData\Local\Temp\ipykernel_13188\2502262557.py:61: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython.display
  from IPython.core.display import display, HTML


# Instalación del paquete nltk

In [40]:
import nltk

nltk.download('conll2002')

nltk.corpus.conll2002.fileids()

[nltk_data] Downloading package conll2002 to
[nltk_data]     C:\Users\Univalle\AppData\Roaming\nltk_data...
[nltk_data]   Package conll2002 is already up-to-date!


['esp.testa', 'esp.testb', 'esp.train', 'ned.testa', 'ned.testb', 'ned.train']

In [47]:
%%time
##biobert

bio_train_sents = list(biobert_dataset['train'])

bio_test_sents = list(biobert_dataset['test'])

bio_eval_sents = list(biobert_dataset['validation'])

print(len(bio_train_sents),len(max(bio_train_sents,key=len)))

print(len(bio_test_sents),len(max(bio_test_sents,key=len)))

print(len(bio_eval_sents),len(max(bio_eval_sents,key=len)))

9788 2
2496 2
2758 2
CPU times: total: 156 ms
Wall time: 404 ms


In [50]:
print(bio_train_sents[0])

{'sentencia': ['Abuela', 'materna', 'con', 'cancer', 'de', 'mama', 'a', 'los', '70', 'años', '.'], 'tag': [4, 19, 29, 0, 16, 16, 29, 29, 10, 8, 29]}


In [41]:
!pip install datasets

     ---------------------------------------- 0.0/480.6 kB ? eta -:--:--
     ------------------------------------- 480.6/480.6 kB 14.7 MB/s eta 0:00:00
     ---------------------------------------- 0.0/179.3 kB ? eta -:--:--
     ---------------------------------------- 179.3/179.3 kB ? eta 0:00:00
     ---------------------------------------- 0.0/25.1 MB ? eta -:--:--
     -- ------------------------------------- 1.9/25.1 MB 57.6 MB/s eta 0:00:01
     -------- ------------------------------- 5.3/25.1 MB 67.1 MB/s eta 0:00:01
     -------------- ------------------------- 9.2/25.1 MB 83.8 MB/s eta 0:00:01
     -------------------- ------------------ 13.3/25.1 MB 93.0 MB/s eta 0:00:01
     ------------------------- ------------ 16.9/25.1 MB 108.8 MB/s eta 0:00:01
     --------------------------------- ---- 21.8/25.1 MB 108.8 MB/s eta 0:00:01
     -------------------------------------  25.1/25.1 MB 108.8 MB/s eta 0:00:01
     -------------------------------------  25.1/25.1 MB 108.8 MB/s


[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [43]:
from datasets import load_dataset

biobert_dataset = load_dataset('Biobert_json.py', trust_remote_code=True)

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [46]:
len(biobert_dataset['train'])

9788

# PARTE  1. PREPROCESAMIENTO DE LOS DATOS

In [8]:
def sent2labels(sent):

    return [label for token, postag, label in sent]



def sent2tokens(sent):

    return [token for token, postag, label in sent]

In [9]:
sent2tokens(train_sents[0])[0]

#sent2labels(train_sents[0])[0]

'Melbourne'

In [52]:
%%time
##biobert
X_bio_train = [s['sentencia'] for s in bio_train_sents]

y_bio_train = [s['tag'] for s in bio_train_sents]



X_bio_test = [s['sentencia'] for s in bio_test_sents]

y_bio_test = [s['tag'] for s in bio_test_sents]



X_bio_eval = [s['sentencia'] for s in bio_eval_sents]

y_bio_eval = [s['tag'] for s in bio_eval_sents]

CPU times: total: 0 ns
Wall time: 0 ns


In [53]:
print(X_bio_train[2])

print(y_bio_train[2])

['-', 'Quiste', 'renal', 'izquierdo', 'complicado', '(', 'ecografia', 'noviembre', '2013', 'quistes', 'renales', 'bilaterales', ')', '.']
[29, 29, 29, 29, 29, 29, 29, 2, 17, 29, 29, 29, 29, 29]


In [54]:
##biobert
import numpy as np



bio_words, bio_tagsss = set([]), set([])

 

for s in (X_bio_train + X_bio_eval + X_bio_test):

    for w in s:

        bio_words.add(w.lower())



for ts in (y_bio_train + y_bio_eval + y_bio_test):

    for t in ts:

        bio_tagsss.add(t)



bio_word2index = {w: i + 2 for i, w in enumerate(list(bio_words))}

bio_word2index['-PAD-'] = 0  # The special value used for padding

bio_word2index['-OOV-'] = 1  # The special value used for OOVs

 

bio_tag2index = {t: i + 2 for i, t in enumerate(list(bio_tagsss))}

bio_tag2index['-PAD-'] = 0  # The special value used to padding

bio_tag2index['-OOV-'] = 1  # The special value used to padding



print (len(bio_word2index))

print (len(bio_tag2index))



np.save("outputs/bio_word2index.npy", bio_word2index)

np.save("outputs/bio_tag2index.npy", bio_tag2index)

print(bio_tagsss)

9883
32
{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29}


In [56]:
##biobert
bio_train_sentences_X, bio_eval_sentences_X, bio_test_sentences_X, bio_train_tags_y, bio_eval_tags_y, bio_test_tags_y = [], [], [], [], [], []



for s in X_bio_train:

    s_int = []

    for w in s:

        try:

            s_int.append(bio_tag2index[w.lower()])

        except KeyError:

            s_int.append(bio_word2index['-OOV-'])

 

    bio_train_sentences_X.append(s_int)



for s in X_bio_eval:

    s_int = []

    for w in s:

        try:

            s_int.append(bio_word2index[w.lower()])

        except KeyError:

            s_int.append(bio_word2index['-OOV-'])

 

    bio_eval_sentences_X.append(s_int)



for s in X_bio_test:

    s_int = []

    for w in s:

        try:

            s_int.append(bio_word2index[w.lower()])

        except KeyError:

            s_int.append(bio_word2index['-OOV-'])

 

    bio_test_sentences_X.append(s_int)



for s in y_bio_train:

    s_int = []

    for w in s:

        try:

            s_int.append(bio_tag2index[w])

        except KeyError:

            s_int.append(bio_tag2index['-OOV-'])

            

    bio_train_tags_y.append(s_int)



for s in y_bio_eval:

    s_int = []

    for w in s:

        try:

            s_int.append(bio_tag2index[w])

        except KeyError:

            s_int.append(bio_tag2index['-OOV-'])

            

    bio_eval_tags_y.append(s_int)



for s in y_bio_test:

    s_int = []

    for w in s:

        try:

            s_int.append(bio_tag2index[w])

        except KeyError:

            s_int.append(bio_tag2index['-OOV-'])

            

    bio_test_tags_y.append(s_int)


# Las matrices de los tags son de números indexados pequeños porque solo son 11 tags.  ({ORG, LOC, PER}  X IOB)

In [57]:
##biobert
print("Longitudes de las Matrices:")

print(len(bio_train_sentences_X))

print(len(bio_eval_sentences_X))

print(len(bio_test_sentences_X))

print(len(bio_train_tags_y))

print(len(bio_eval_tags_y))

print(len(bio_test_tags_y))



print("\nMuestra de Datos presentes en las Matrices con las transformaciones:\n")





print(bio_train_sentences_X[0])

print(bio_eval_sentences_X[0])

print(bio_test_sentences_X[0])

print(bio_train_tags_y[0])

print(bio_eval_tags_y[0])

print(bio_test_tags_y[0])


Longitudes de las Matrices:
9788
2758
2496
9788
2758
2496

Muestra de Datos presentes en las Matrices con las transformaciones:

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[5160, 2455, 4788, 3023, 4902, 5502, 926, 7482, 656, 4339, 7292, 2442, 3830, 7360, 159, 9527]
[8394, 3440, 7923, 5968, 6789, 9259, 590, 4275, 8491, 6019, 9527]
[6, 21, 31, 2, 18, 18, 31, 31, 12, 10, 31]
[10, 12, 5, 10, 12, 31, 5, 12, 10, 31, 10, 31, 7, 22, 22, 31]
[31, 31, 12, 10, 31, 31, 31, 31, 31, 4, 31]


# Se procede a Normalizar las matrices con la longitud de la columna=MAX_LENGTH1 para que todas contengan el mismo numero de columnas, con la longitud máxima de palabras encontradas anteriormente y se agregan ceros a la derecha en las posiciones que hacen falta en el vector. 

In [58]:
##biobert



MAX_LENGTH=202 

bio_train_sentences_X = pad_sequences(bio_train_sentences_X, maxlen=MAX_LENGTH, padding='post')

bio_eval_sentences_X = pad_sequences(bio_eval_sentences_X, maxlen=MAX_LENGTH, padding='post')

bio_test_sentences_X = pad_sequences(bio_test_sentences_X, maxlen=MAX_LENGTH, padding='post')

bio_train_tags_y = pad_sequences(bio_train_tags_y, maxlen=MAX_LENGTH, padding='post')

bio_eval_tags_y = pad_sequences(bio_eval_tags_y, maxlen=MAX_LENGTH, padding='post')

bio_test_tags_y = pad_sequences(bio_test_tags_y, maxlen=MAX_LENGTH, padding='post')

 

print(bio_train_sentences_X[0])

print(bio_train_sentences_X.shape)

print(bio_eval_sentences_X[0])

print(bio_eval_sentences_X.shape)

print(bio_test_sentences_X[0])

print(bio_test_sentences_X.shape)

print(bio_train_tags_y[0])

print(bio_train_tags_y.shape)

print(bio_eval_tags_y[0])

print(bio_eval_tags_y.shape)

print(bio_test_tags_y[0])

print(bio_test_tags_y.shape)



np.save("outputs/bio_train_sentences_X.npy", bio_train_sentences_X)

np.save("outputs/bio_eval_sentences_X.npy", bio_eval_sentences_X)

np.save("outputs/bio_test_sentences_X.npy", bio_test_sentences_X)

np.save("outputs/bio_train_tags_y.npy", bio_train_tags_y)

np.save("outputs/bio_eval_tags_y.npy", bio_eval_tags_y)

np.save("outputs/bio_test_tags_y.npy", bio_test_tags_y)

[1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
(9788, 202)
[5160 2455 4788 3023 4902 5502  926 7482  656 4339 7292 2442 3830 7360
  159 9527    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    

In [59]:
def to_categoricals(sequences, categories):

    cat_sequences = []

    for s in sequences:

        cats = []

        for item in s:

            cats.append(np.zeros(categories))

            cats[-1][item] = 1.0

        cat_sequences.append(cats)

    return np.array(cat_sequences)

In [60]:




def encode(data):

    print('Shape of data (BEFORE encode): %s' % str(data.shape))

    encoded = to_categorical(data)

    print('Shape of data (AFTER  encode): %s\n' % str(encoded.shape))

    return encoded

# Se realiza la categorización one-hot de las etiquetas o labels de entrenamiento, testeo y validación

In [61]:
##biobert

bio_cat_train_tags_y = to_categoricals(bio_train_tags_y, len(bio_tag2index))

bio_cat_eval_tags_y  = to_categoricals(bio_eval_tags_y, len(bio_tag2index))

bio_cat_test_tags_y  = to_categoricals(bio_test_tags_y, len(bio_tag2index))



print(bio_cat_train_tags_y[1])

print(len(bio_cat_train_tags_y))

print(len(bio_cat_test_tags_y))

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 1.]
 ...
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]]
9788
2496


# PARTE 2. ENTRENAMIENTO DEL MODELO DE RED.

In [20]:


EMBED_DIM=300

file = 'inputs/ccfasttext/cc.es.300.vec'

embedding_matrix = bme(file, len(word2index), EMBED_DIM, word2index)

Cargando archivo...


0it [00:00, ?it/s]

Encontrado 2000000 Word Vectors.


  0%|          | 0/28384 [00:00<?, ?it/s]

Convertidos: 25347 Tokens | Perdidos: 3445 Tokens


In [21]:
import tensorflow as tf
import keras

print("TensorFlow version:", tf.__version__)
print("Keras version:", keras.__version__)

TensorFlow version: 2.12.1
Keras version: 2.12.0


In [62]:
##biobert
from tf2crf import CRF as crf6

from mwrapper import ModelWithCRFLoss, ModelWithCRFLossDSCLoss

from utils import build_matrix_embeddings as bme, plot_model_performance, logits_to_tokens, report_to_df

from tensorflow.keras.layers import Concatenate, Lambda, Input, LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional, InputLayer, Activation, Flatten, Masking

from tensorflow.keras.optimizers import Adam, schedules

input = Input(shape=(MAX_LENGTH,))

word_embedding_size = 300



# Embedding Layer

#model = Embedding(input_dim=len(word2index), 

    #            output_dim=word_embedding_size, 

     #           input_length=MAX_LENGTH,

     #           mask_zero=False)(input)



model = Embedding(len(bio_word2index),

                        EMBED_DIM,

                        input_length=MAX_LENGTH,  

                        #weights=[embedding_matrix],

                        trainable=False,

                        mask_zero=True)(input)



# BI-LSTM Layer

model = Bidirectional(LSTM(units=word_embedding_size, 

                     return_sequences=True, 

                     dropout=0.5, 

                     recurrent_dropout=0.5))(model)

model  = Dropout(0.5, name='dropout_lstm')(model)

model  = Dense(units=EMBED_DIM * 2, activation='relu')(model)

model  = Dense(units=len(bio_tag2index), activation='relu')(model)

    

model  = Masking(mask_value=0.,input_shape=(MAX_LENGTH, len(bio_tag2index)))(model)

    

crf = crf6(units=len(bio_tag2index), name="ner_crf")

predictions = crf(model)



base_model = Model(inputs=input, outputs=predictions)

model = ModelWithCRFLoss(base_model, sparse_target=True)

    

model.compile(optimizer='adam')

In [ ]:
#biobert
history= model.fit(bio_train_sentences_X, bio_cat_train_tags_y,

                       validation_data=(bio_eval_sentences_X, bio_cat_eval_tags_y),

                       batch_size=64, 

                       epochs=20,

                       verbose=2)

Epoch 1/20
153/153 - 592s - loss: 20.0810 - accuracy: 0.9783 - val_loss_val: 86.3857 - val_val_accuracy: 0.9217 - 592s/epoch - 4s/step
Epoch 2/20


In [30]:
print(tag2index)

y_pred= model.predict(test_sentences_X)

print(y_pred.shape)

{'I-LOC': 2, 'B-MISC': 3, 'I-ORG': 4, 'B-LOC': 5, 'B-ORG': 6, 'I-PER': 7, 'B-PER': 8, 'O': 9, 'I-MISC': 10, '-PAD-': 0, '-OOV-': 1}
48/48 [==============================] - 37s 769ms/step
(1517, 202)


In [31]:
from utils import build_matrix_embeddings as bme, plot_model_performance, logits_to_tokens, report_to_df

index2tag = {i: t for t, i in tag2index.items()}

print(index2tag)

y1_pred = logits_to_tokens(y_pred, index2tag)

print(y1_pred[10])

{2: 'I-LOC', 3: 'B-MISC', 4: 'I-ORG', 5: 'B-LOC', 6: 'B-ORG', 7: 'I-PER', 8: 'B-PER', 9: 'O', 10: 'I-MISC', 0: '-PAD-', 1: '-OOV-'}
['O', 'O', 'O', 'O', 'O', 'B-ORG', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-',

In [32]:
#print(Y_test[4])

print(test_tags_y.shape)

(1517, 202)


In [33]:
from utils import build_matrix_embeddings as bme, plot_model_performance, logits_to_tokens, report_to_df

index2tag = {i: t for t, i in tag2index.items()}

print(index2tag)

y1_true = logits_to_tokens(test_tags_y, index2tag)

print(y1_true[10])

{2: 'I-LOC', 3: 'B-MISC', 4: 'I-ORG', 5: 'B-LOC', 6: 'B-ORG', 7: 'I-PER', 8: 'B-PER', 9: 'O', 10: 'I-MISC', 0: '-PAD-', 1: '-OOV-'}
['O', 'O', 'O', 'O', 'O', 'B-ORG', 'I-ORG', 'I-ORG', 'I-ORG', 'O', 'O', 'O', 'O', 'O', 'B-ORG', 'I-ORG', 'O', 'O', 'B-ORG', 'I-ORG', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-'

In [ ]:
!pip install seqeval

In [34]:
#hh1 = seqclarep(results['Expected'], results['Predicted'])

#print('\nclassification_report:\n', hh1)

from seqeval.metrics import classification_report as seqclarep

from seqeval.metrics import precision_score, recall_score, f1_score, accuracy_score

print("precision: {:.1%}".format(precision_score(y1_true, y1_pred)))

print("   recall: {:.1%}".format(recall_score(y1_true,    y1_pred)))

print(" accuracy: {:.1%}".format(accuracy_score(y1_true,  y1_pred)))

print(" F1-score: {:.1%}".format(f1_score(y1_true,        y1_pred)))

precision: 75.3%
   recall: 28.0%
 accuracy: 98.4%
 F1-score: 40.9%


In [ ]:
import pandas as pd

li1 = sum(y1_true, [])

li2 = sum(y1_pred, [])



results = pd.DataFrame(columns=['Expected', 'Predicted'])



results['Expected'] = li1

results['Predicted'] = li2

In [ ]:
from sklearn.metrics import classification_report as eskclarep

report = eskclarep(results['Expected'], results['Predicted'])

#print('\nclassification_report:\n', report)



print(report_to_df(report))

In [ ]:
test_samples = [

    "James Rodriguez es el jugador colombiano más importante con Radamel Falcao.".split(),

    " Jugadores de la selección Colombia que juegan en el Reino Unido".split()

]

#print(max(test_samples))

print(test_samples)

In [ ]:
test_samples_X = []

for s in test_samples:

    s_int = []

    for w in s:

        try:

            s_int.append(word2index[w.lower()])

        except KeyError:

            s_int.append(word2index['-OOV-'])

    test_samples_X.append(s_int)



test_samples_X = pad_sequences(test_samples_X, maxlen=MAX_LENGTH, padding='post')

print(test_samples_X)

print(test_samples_X.shape)

In [ ]:
predictions = model.predict(test_samples_X)

print(predictions, predictions.shape)

In [ ]:
#print(len(predictions))

log_tokens = logits_to_tokens(predictions, {i: t for t, i in tag2index.items()})

print(log_tokens)

In [ ]:
#!pip install tabulate

from tabulate import tabulate



heads1 = test_samples[0]

body1 = [log_tokens[0][:len(test_samples[0])]]



heads2 = test_samples[1]

body2 = [log_tokens[1][:len(test_samples[1])]]



print(tabulate(body1, headers=heads1))



print ("\n")



print(tabulate(body2, headers=heads2))





## postagging Freeling 4.1



## El      hombre   bajo     corre    bajo  el      puente   con  bajo  índice   de  adrenalina  .

## DA0MS0  NCMS000  AQ0MS00  VMIP3S0  SP    DA0MS0  NCMS000  SP   SP    NCMS000  SP  NCFS000     Fp





## pos tagger Stanford NLP



## El      hombre   bajo     corre    bajo  el      puente   con    bajo   índice  de    adrenalina  .

## da0000  nc0s000  aq0000   vmip000  sp000 da0000  nc0s000  sp000  aq0000 nc0s000 sp000 nc0s000     fp